In [ ]:
import os
import matplotlib.pyplot as plt
import h5py
import numpy as np
import networkx as nx
import yaml

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "font.size": 12,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directory ----------------------
# here is the path to the results directory
# ---------------------------------------------------------------------------
results_dir_path = f"{script_dir}/outputs/qbp/2024-05-06/09-20-04"
results_path = f"{results_dir_path}/result"
config_path = f"{results_dir_path}/.hydra/config.yaml"
with open(config_path, "r") as f:
    raw_config = f.read()
    config = yaml.safe_load(raw_config)
sample_measurements = bool(config["quantum_annealing_schedule"]["sample_measurements"])
layers_number = int(config["quantum_annealing_schedule"]["steps_number"])
with h5py.File(results_path, "r") as f:
    circuit = nx.Graph()
    fields = np.array(f["fields"])
    pairs = np.array(f["coupled_spin_pairs"])
    coupling_amplitudes = np.array(f["coupling_amplitudes"])
    truncation_errors = np.array(f["truncation_errors"])
    entropies = np.array(f["entropies"])
    quantum_annealing_color_wo_sampling = list(map(lambda x: "blue" if x > 0.5 else "red", np.array(f["density_matrices"])[:, 0, 0]))
    quantum_annealer_energy_wo_sampling =  np.array(f["quantum_annealer_energy_wo_sampling"])
    if sample_measurements:
        quantum_annealer_coloring = list(map(lambda x: "blue" if x > 0 else "red", np.array(f["quantum_annealer_configuration"])))
        quantum_annealer_energy = np.array(f["quantum_annealer_energy"])
    vidal_distances_after_regauging = np.array(f["vidal_distances_after_regauging"])
    for id in range(fields.shape[0]):
        circuit.add_node(id)
    for pair in pairs:
        circuit.add_edge(pair[0], pair[1])
    fig_graph_wo_sampling, ax = plt.subplots(1, 1)
    ax.set_title(r"{\rm Energy = " + str(quantum_annealer_energy_wo_sampling) + r"}")
    nx.draw_kamada_kawai(circuit, node_size=5, width=1., node_color=quantum_annealing_color_wo_sampling, ax=ax)
    if sample_measurements:
        fig_graph_with_sampling, ax = plt.subplots(1, 1)
        ax.set_title(r"{\rm Energy = " + str(quantum_annealer_energy) + r"}")
        nx.draw_kamada_kawai(circuit, node_size=5, width=1., node_color=quantum_annealer_coloring, ax=ax)
    fig_truncation_error, ax = plt.subplots(1, 1)
    ax.plot(truncation_errors, 'b')
    ax.set_ylabel(r"{\rm Error}")
    ax.set_xlabel(r"{\rm Two-qubit gate number}")
    ax.set_yscale('log')
    fig_vidal_dist, ax = plt.subplots(1, 1)
    ax.plot(vidal_distances_after_regauging, 'b')
    ax.set_ylabel(r"{\rm Vidal distance}")
    ax.set_xlabel(r"{\rm Regauging number}")
    fig_entropy, ax = plt.subplots(1, 1)
    ax.plot(entropies, 'b')
    ax.set_ylabel(r"{\rm Entropy}")
    ax.set_xlabel(r"{\rm Two-qubit gate number}")

In [ ]:
fig_graph_wo_sampling.savefig(f"{results_dir_path}/qbp_graph_wo_sampling.pdf", bbox_inches="tight")
fig_truncation_error.savefig(f"{results_dir_path}/qbp_truncation_error.pdf", bbox_inches="tight")
fig_vidal_dist.savefig(f"{results_dir_path}/qbp_vidal_distance.pdf", bbox_inches="tight")
fig_entropy.savefig(f"{results_dir_path}/qbp_entropy.pdf", bbox_inches="tight")
if sample_measurements:
    fig_graph_with_sampling.savefig(f"{results_dir_path}/qbp_graph_with_sampling.pdf", bbox_inches="tight")